#### CT-208 - Matemática Computacional
##### Paper Model

In [16]:
import os
import re
import numpy as np

from PIL             import Image
from parseSvgFiles   import parseSVG
from convertSvg2Png  import convertSVG2PNG
from convertPng2Svg  import convertPNG2SVG
from placeComponent  import ComponentPlacing
from utils           import binarray2jpg, png2bin, selectComponentToPlace


##### 1. Parse SVG files

Parsing each SVG file into separate components

In [17]:
# Select figure in folder and parse SVG file into separate components 
for svg_name in ['example', 'harry-potter', 'e190-e2']:
    parseSVG(svg_name)

Components from example separated with success!
Components from harry-potter separated with success!
Components from e190-e2 separated with success!


##### 2. Convert SVG components

Convert each SVG component into a PNG equivalent

In [18]:
# Parse each SVG component into PNG equivalent 
for svg_name in ['example', 'harry-potter', 'e190-e2']:
    convertSVG2PNG(svg_name)

Components from example transformed with success!
Components from harry-potter transformed with success!
Components from e190-e2 transformed with success!


In [19]:
# Conversion parameters of each project
params = dict()
params['example'] = {'MM2PX': 1, 'PX2MM': 1}
params['harry-potter'] = {'MM2PX': 3.7795276032851057, 'PX2MM': 0.2645833302370423}
params['e190-e2'] = {'MM2PX': 1, 'PX2MM': 1}

# Sheet size
PAGE_SIZE = (210,297) 

In [20]:
# Define which project to optimize
svg_name = 'example'
#svg_name = 'harry-potter'
#svg_name = 'e190-e2'

# Conversion parameters
MM2PX = params[svg_name]['MM2PX']
PX2MM = params[svg_name]['PX2MM']

# Components and file paths
components_path = './' + svg_name + '/components_png'
test_results_path = './' + svg_name + '/test_results'

print("Conversion MM2PX: " + str(MM2PX))
print("Conversion PX2MM: " + str(PX2MM))
print("\nPaths:")
print(components_path)
print(test_results_path)

Conversion MM2PX: 1
Conversion PX2MM: 1

Paths:
./example/components_png
./example/test_results


In [21]:
# Finds the component png file in the directory
files = os.listdir(components_path)
pngs = list(filter(lambda name: re.match('.*\.png', name), files))

In [22]:
# Optimizer parameters
iterations = 300
sigma = 1.0
population = 12

In [23]:
# Creates a list of pages and append a page of (height, lenght) px
pages = []
pages.append(np.zeros(PAGE_SIZE, dtype=int))
# Transforms each png component into a binary matrix
# List of binary matrix of components and its name
components = [[png2bin(components_path + '/' + png), png] for png in pngs] 
# Creates a list of objects of class ComponentPlacing
CP_list = [ComponentPlacing(pages[0], component, sigma=sigma, population=population, num_iterations=iterations) for component in components]


(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=364948, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=299456, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=394472, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=344748, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=361091, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=331526, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=313773, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=303051, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=361677, Sun Jun 26 22:22:45 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=383229, Sun Jun 26 22:22:45 2022)


In [24]:
# Place components on page
results = []
page_idx = 0
max_attempts = 100
num_components = len(CP_list)


In [25]:
while CP_list:
    attempt = 0     # counter of times that try to fit component in a page
    # This loop tries to fit a component on current page    
    while attempt < max_attempts:
        # Selects one aleatory component
        idx = selectComponentToPlace(CP_list)
        CP_object = CP_list[idx]
        # Finds the best position
        CP_object.FindCandidatePosition()
        # Checks if there is no intersection and place the compoponent if not
        if CP_object.NoIntersection():
            pages[page_idx] = CP_object.PlaceComponentInPage()
            # removes the placed component from the list and store in results list
            results.append(CP_list.pop(idx)) 
            num_components -= 1
            print(('Current page: ' + str(page_idx+1)).ljust(18) + 'Components remaining: ' + str(num_components)+'...')
            
            if CP_list: 
                attempt = 0
            else: 
                break
        else: 
            attempt += 1
            CP_object.resetParameters() # sorts a new initial point
            
    # Prepares a new page if necessary
    if CP_list:
        pages.append(np.zeros(PAGE_SIZE, dtype=int))
        page_idx += 1
        for CP_object in CP_list:
            # sorts new initial point for the objects
            CP_object.page = pages[page_idx]
            CP_object.page_idx = page_idx
            CP_object.resetParameters()


Current page: 1   Components remaining: 9...
Current page: 1   Components remaining: 8...
Current page: 1   Components remaining: 7...
Current page: 1   Components remaining: 6...
Current page: 1   Components remaining: 5...
Current page: 1   Components remaining: 4...
Current page: 1   Components remaining: 3...
Current page: 1   Components remaining: 2...
Current page: 1   Components remaining: 1...
Current page: 1   Components remaining: 0...


In [26]:
# Remove all png files from the folder before storing the results
files = os.listdir(test_results_path)
for file in files:
    os.remove(test_results_path + '/' + file)


In [27]:

# Saves pages
for idx,page in enumerate(pages):
    image = binarray2jpg(page).astype(np.uint8)
    image = Image.fromarray(image)
    image.save(test_results_path + '/page' + str(idx).zfill(2) + '.png')

print('Finished optimization.')

Finished optimization.


In [28]:
# Parse each PNG file into SVG file 
for svg_name in ['example', 'harry-potter', 'e190-e2']:
    convertPNG2SVG(svg_name)

ModuleNotFoundError: No module named 'place_component'